# Example 06 - WidebandSig53 with YOLOv8 Detector
This notebook showcases using the WBSig53 dataset to train a YOLOv8 model.

---

## Import Libraries
We will import all the usual libraries, in addition to Ultralytics. You can install Ultralytics with:
```bash
pip install ultralytics
```

In [ ]:
# Packages Imports for Training
from torchsig.utils.yolo_train import *
from datetime import datetime
import yaml

In [ ]:
# Package Imports for Testing/Inference
from torchsig.datasets.datamodules import WidebandSig53DataModule
from torchsig.transforms.transforms import Spectrogram, SpectrogramImage, Normalize, Compose, Identity
from torchsig.transforms.target_transforms import DescToBBoxFamilyDict
from ultralytics import YOLO
from PIL import Image

-----------------------------
## Check or Generate the Wideband Sig53 Dataset
To generate the WidebandSig53 dataset, several parameters are given to the imported `WidebandSig53DataModule` class. These paramters are:
- `root` ~ A string to specify the root directory of where to generate and/or read an existing WidebandSig53 dataset
- `train` ~ A boolean to specify if the WidebandSig53 dataset should be the training (True) or validation (False) sets
- `qa` - A boolean to specify whether to generate a small subset of Sig53 (True), or the full dataset (False), default is True
- `impaired` ~ A boolean to specify if the WidebandSig53 dataset should be the clean version or the impaired version
- `transform` ~ Optionally, pass in any data transforms here if the dataset will be used in an ML training pipeline. Note: these transforms are not called during the dataset generation. The static saved dataset will always be in IQ format. The transform is only called when retrieving data examples.
- `target_transform` ~ Optionally, pass in any target transforms here if the dataset will be used in an ML training pipeline. Note: these target transforms are not called during the dataset generation. The static saved dataset will always be saved as tuples in the LMDB dataset. The target transform is only called when retrieving data examples.

A combination of the `train` and the `impaired` booleans determines which of the four (4) distinct WidebandSig53 datasets will be instantiated:
| `impaired` | `qa` | Result |
| ---------- | ---- | ------- |
| `False` | `False` | Clean datasets of train=250k examples and val=25k examples |
| `False` | `True` | Clean datasets of train=250 examples and val=250 examples |
| `True` | `False` | Impaired datasets of train=250k examples and val=25k examples |
| `True` | `True` | Impaired datasets of train=250 examples and val=250 examples |

The final option of the impaired validation set is the dataset to be used when reporting any results with the official WidebandSig53 dataset.

In [ ]:
# Generate WidebandSig53 DataModule
root = "../datasets/wideband_sig53"
impaired = True
qa = True
fft_size = 512
num_classes = 53
batch_size = 1

transform = Compose([    
])

target_transform = Compose([
    DescToBBoxFamilyDict()
])

datamodule = WidebandSig53DataModule(
    root=root,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=transform,
    target_transform=target_transform,
    batch_size=batch_size
)

In [ ]:
datamodule.prepare_data()
datamodule.setup("fit")

wideband_sig53 = datamodule.train

# Retrieve a sample and print out information
idx = np.random.randint(len(wideband_sig53))
data, label = wideband_sig53[idx]
print("Dataset length: {}".format(len(wideband_sig53)))
print("Data shape: {}".format(data.shape))
print("Label: {}".format(label))

## Prepare YOLO trainer and Model
Next, the datasets are rewritten to disk that is Ultralytics YOLO compatible. See [Ultralytics: Train Custom Data - Organize Directories](https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#23-organize-directories) to learn more. 

Additionally, create a yaml file for dataset configuration. See [Ultralytics: Train Custom Data - Create dataset.yaml](https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#21-create-datasetyaml)

Download desired YOLO model from [Ultralytics Models](https://docs.ultralytics.com/models/). We will use YOLOv8, specifically `yolov8n.pt`

---

### Explanation of the `overrides` Dictionary

The `overrides` dictionary is used to customize the settings for the Ultralytics YOLO trainer by specifying specific values that override the default configurations. The dictionary is imported from `wbdata.yaml`. However, you can customize in the notebook. 

Example:

```python
overrides = {'model': 'yolov8n.pt', 'epochs': 100, 'data': 'wbdata.yaml', 'device': 0, 'imgsz': 512, 'single_cls': True}
```
A .yaml is necessary for training. Look at `wbdata.yaml` in the examples directory. It will contain the path to your torchsig data.


### Dataset Location Warning

There must exist a datasets directory at `/path/to/torchsig/datasets`.

This example assumes that you have generated `train` and `val` lmdb wideband datasets at `../datasets/wideband_sig53/`

You can also specify an absolute path to your dataset in `wbdata.yaml`.

In [ ]:
with open('wbdata.yaml', 'r') as file:
    config = yaml.safe_load(file)
overrides = config['overrides']
print(f"Creating experiment -> {overrides['name']}")

In [ ]:
trainer = Yolo_Trainer(overrides=overrides)

## Train
Train YOLO. See [Ultralytics Train](https://docs.ultralytics.com/modes/train/#train-settings) for training hyperparameter options.

---

In [ ]:
trainer.train()

## Evaluation
Check model performance from training. From here, you can use the trained model to test on prepared data (numpy image arrays of spectrograms)

Will load example from Torchsig

model_path is path to best.pt from your training session. Path is printed at the end of training.

---

## Generate and Instantiate WBSig53 Test Dataset
After generating the WBSig53 dataset (see `03_example_widebandsig53_dataset.ipynb`), we can instantiate it with the needed transforms. Change `root` to test dataset path.

---

In [ ]:
test_path = '../datasets/wideband_sig53_test' #Should differ from your training dataset

transform = Compose([
    Spectrogram(nperseg=512, noverlap=0, nfft=512, mode='psd'),
    Normalize(norm=np.inf, flatten=True),
    SpectrogramImage(), 
    ])

test_data = WidebandSig53DataModule(
    root=test_path,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=transform,
    target_transform=None,
    batch_size=batch_size
)

test_data.prepare_data()
test_data.setup("fit")

wideband_sig53_test = test_data.train

# Retrieve a sample and print out information
idx = np.random.randint(len(wideband_sig53_test))
data, label = wideband_sig53_test[idx]
print("Dataset length: {}".format(len(wideband_sig53_test)))
print("Data shape: {}".format(data.shape))

samples = []
labels = []
for i in range(10):
    idx = np.random.randint(len(wideband_sig53_test))
    sample, label = wideband_sig53_test[idx]
    lb = [l['class_name'] for l in label]
    samples.append(sample)
    labels.append(lb)

### Load model 
The model path is printed after training. Use the best.pt weights

In [ ]:
model_path = 'YOUR_PROJECT_NAME/YOUR_EXPERIMENT_NAME/weights/best.pt'  #Double check this is correct path, printed after training.
model = YOLO(model_path)

In [ ]:
# Inference will be saved to path printed after predict. 
results = model.predict(samples, save=True, imgsz=512, conf=0.5)

In [ ]:
# Process results list
for y, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    probs = result.probs  # Probs object for classification outputs
    print(f'Actual Labels -> {labels[y]}')
    result.show()  # display to screen